# Лабораторная работа 3. Компьютерное моделирование. Модели, сводящиеся к СЛАУ.
Написать подпрограммы решения систем линейных алгебраических уравнений на
языках Python или Scilab:
1. методом Гаусса;
2. методом простой итерации;
3. методом Зейделя;
4. метод наискорейшего спуска;
5. метод минимальных невязок;
6. метод сопряженных градиентов.

Протестировать разработанные подпрограммы при решении следующих задач:11,1,7

1.Из некоторого листового материала необходимо выкроить 360 заготовок типа А, 300 заготовок типа Б и 675 заготовок типа В. При этом можно применять три способа раскроя. При первом способе раскроя получается 3 заготовки типа А, 1 заготовка типа Б и 4 заготовки типа В, при втором способе раскроя получается 2 заготовки типа А, 6 заготовок типа Б и 1 заготовка типа В, при третьем способе раскроя получается 1 заготовка типа А, 2 заготовки типа Б и 5 заготовок типа В. Найти расход материала при каждом из указанных способов раскроя.

Для решения данной задачи выпишим способы раскроя при помощи переменных x,y,z.
x: 3A+1B+4C;
y: 2A+6B+1C;
z: 1A+2B+5C;

Составим систему уравнений 
\begin{equation*}
 \begin{cases}
 3x+2y+z=360\\
 x+6y+2z=300\\
 4x+y+5z=675
 \end{cases}
\end{equation*}

#### Метод Гаусса

Для решения системы линейных уравнений методом Гаусса необходимо выполнить следующие шаги:

1. Записать систему линейных уравнений в виде матрицы коэффициентов при неизвестных.
2. Выполнить элементарные преобразования над строками матрицы для получения так называемого ступенчатого вида.
3. Найти решения полученной системы и, при необходимости, проверить их на непротиворечивость.

In [28]:
import math
def gaussian(A, b):
    '''
    Приминает на вход две переменные:
    A - матрица коэффициентов - квадратный двумерный numpy array типа float
    b - вектор свободных членов - numpy array типа float
    Вовращает вектор решений
    '''

    # составляем расширенную матрицу системы
    reshaped_b = b.reshape((len(b), 1))
    A = np.hstack((A, reshaped_b))
    
    # приводим матрицу к треугольному виду
    # i - опорная строка
    # j - текущая строка (всегда меньше i)
    for i in range(len(A)):
        for j in range(i + 1, len(A)):
            A[j] -= A[i] * A[j][i] / A[i][i]
            
    # обратный ход
    x = np.array([0] * len(b), dtype=float) # вектор решений
    i = len(A) - 1
    while i >= 0:
        x[i] = (A[i][-1] - sum(x * A[i][0:-1])) / A[i][i] 
        i -= 1
        
    return x


A_list =[[
    [3, 2, 1],
    [1, 6, 2],
    [4, 1, 5]]]          
          
b_list = [ [360, 300, 675] ]
for A in A_list:
    A = np.array(A, dtype=float)
    
for b in b_list:
    b = np.array(b, dtype=float)
for i in range(len(A_list)):
    
    print('Решение методом Гаусса:')
    print(gaussian(A, b), '\n')

Решение методом Гаусса:
[90. 15. 60.] 



#### Метод итерации

Порядок решения СЛАУ методом Якоби (простых итераций) такой:
1. Приведение системы уравнений к виду, в котором на каждой строчке выражено какое-либо неизвестное значение системы.
2. Произвольный выбор нулевого решения, в качестве него можно взять вектор-столбец свободных членов.
3. Производим подстановку произвольного нулевого решения в систему уравнений, полученную под пунктом 1.
4. Осуществление дополнительных итераций, для каждой из которых используется решение, полученное на предыдущем этапе.

Ограничения:
1. матрица квадратная
2. элементы, лежащие на главное диагонали матрицы не должны быть нулевые.
3. в сумму произведений, не встречается элемента с совпадающими индексами (тот а[k][k] который лежит на диагонали), поэтому при суммировании мы должны его пропустить.

In [46]:
import math
import copy
 
a = [[3, 2, 1],
     [1, 6, 2],
     [4, 1, 5]]
     
b = [360, 300, 675]
 
# Проверка матрицы коэффициентов на корректность
def isCorrectArray(a):
    for row in range(0, len(a)):
        if( len(a[row]) != len(b) ):
            print('Не соответствует размерность')
            return False
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True
 
# Условие завершения программы на основе вычисления
# расстояния между соответствующими элементами соседних
# итераций в методе решения 
def isNeedToComplete(x_old, x_new):
    eps = 0.0001
    sum_up = 0
    sum_low = 0
    for k in range(0, len(x_old)):
        sum_up += ( x_new[k] - x_old[k] ) ** 2
        sum_low += ( x_new[k] ) ** 2
        
    return math.sqrt( sum_up / sum_low ) < eps
 
# Процедура решения
def solution(a, b):
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
    else:
        count = len(b) # количество корней
        
        x = [1 for k in range(0, count) ] # начальное приближение корней
        
        nIter = 0  # подсчет количества итераций
        MAXI = 100    # максимально допустимое число итераций
        while( nIter < MAXI ):
 
            x_prev = copy.deepcopy(x)
            
            for k in range(0, count):
                S = 0
                for j in range(0, count):
                    if( j != k ): S = S + a[k][j] * x[j] 
                x[k] = round(b[k]/a[k][k] - S / a[k][k])
            
            if isNeedToComplete(x_prev, x) : # проверка на выход
                break
              
            nIter += 1
 
        print('Количество итераций на решение: ', nIter)
        
        return x    
                
    
# MAIN - блок программмы
print( 'Решение: ', solution(a, b) ) # Вызываем процедуру решение

Количество итераций на решение:  7
Решение:  [90, 15, 60]


#### Метод Зейделя

Метод Зейделя является итерационным методом решения, то есть находится не точное решение, а некоторое приближение к нему, которое задается пользователем. В свою очередь данный метод является лишь модификацией метода
простой итерации, заключающейся в том, что при вычислении очередного приближения $x^{k+1}$, его уже полученные компоненты ${x_1}^{k+1},...,{x_{i-1}}^{k+1}$ сразу же используются для вычисления ${x_i}^{k+1}$.

In [50]:
import numpy as np

def seidel(m, b, eps):
    n = len(m)
    r = range(n)
    x = [0 for i in r]
    conv = False
    while not conv:
        p = x.copy()
        for i in r:
            var = sum(m[i][j] * x[j] for j in range(i))
            var += sum(m[i][j] * p[j] for j in range(i+1, n))
            x[i] = round((b[i] - var) / m[i][i])
        
        conv = sum((x[i]-p[i])**2 for i in r) < eps
    return x
a = [[3, 2, 1],
     [1, 6, 2],
     [4, 1, 5]]
     
b = [360, 300, 675]

eps = 0.0001

print(seidel(a, b, eps))

[90, 15, 60]


#### Метод наискорейшего спуска
Метод наискорейшего спуска является одной из наиболее фундаментальных процедур минимизации дифференцируемой функции нескольких переменных. Вектор d называется направлением спуска для функции f в точке x, если существует такое d > 0, что f(x+lym*d)<f(x) для всех lym принадлежащих интервалу (0, d). В частности, если

$/frac {lim f(x+ld)}{lym} < 0$ при $lym\rightarrow 0+$

Алгоритм метода наискорейшего спуска

При заданной точке x алгоритм наискорейшего спуска заключается в реализации линейного поиска вдоль направления $ - \frac {grad ((f)x)}{||grad (f(x))||}$ или, что то же самое, вдоль направления -grad(f(x)).

Начальный этап. Пусть eps > 0 - константа остановки. Выбрать начальную точку  , положить  и перейти к основному этапу.

Основной этап. Если $||grad (f(x))||<eps$ , то остановиться; в противном случае положить $dk = -grad(f(x))$ и найти lym[k] - оптимальное решение задачи минимизации

$f(xk+lym*dk)$ при $lym\geq0$ . Положить $x[k+1]=xk=lym[k]*dk , заменить k на k+1 и повторить основной этап.

In [3]:
import numpy as np
from numpy.linalg import norm
from math import *

def steepest_descent_manual(A, b, x_0, max_iters=1000, tol=1e-7, verbose=0):
    x_old = x_0.copy()
    it = 0
    while it < max_iters and norm(A@x_old - b) > tol:
        grad = np.transpose(A.dot(x_old)) - b
        alpha = np.inner(grad, grad) / (np.inner(np.transpose(np.vstack(A).dot(grad)), grad))
        x_new = x_old - alpha * grad
        it += 1
        if verbose >= 1:
            print("Iteration {}: x_old = {}, x_new = {}, error = {}".format(it, x_old, x_new, norm(Ax_old-b)))
        x_old = x_new
        x_old = np.rint(x_old)
    return x_old
A = np.array([[3,2,1],[1,6,2],[4,1,5]])
b = np.array([360,300,675])
x_0 = np.ones(3)
print("\nSolution is: {}".format(steepest_descent_manual(A,b,x_0)))


Solution is: [90. 15. 60.]


#### Метод минимальных невязок
Пусть по-прежнему матрица $A = A^* > 0$. Обозначим через $r^p=Ax^p-f$
невязку, получающуюся при подстановке некоторого значения p x
в уравнение Ax = f. Итерационный алгоритм запишем в виде
    $$x^{p+1}=x^{p-\tau_{p+1}}*r^p, p=0,1,2,3,...$$
где $x^0$ — некоторое начальное приближение.
Методом минимальных невязок называется итерационный метод, в котором $\tau_{p+1}$ выбирается из условия минимума $||r^{p+1}||$
при заданной норме $||r^p||$. Минимум нормы невязки $||r^{p+1}||$
достигается, если
$$\tau_{p+1}=\frac{(Ar^p, r^p)}{||Ar^p||^2}$$
Метод минимальных невязок сходится с той
же скоростью, что и метод простой итерации с оптимальным
параметром.

Метод минимальных невязок по своей идее близок к методу сопряженных градиентов и отличается от последнего
функционалом, который минимизируется на каждом шаге итераций.

In [5]:
import numpy as np

def minimal_residue_method(A, b):
    # Проверка, является ли система уравнений квадратной и не единичной.
    if len(A) != len(b):
        raise ValueError('Input arrays must be square and non-singular.')

    # Вычисляем обратную матрицу A
    A_inv = np.linalg.inv(A)

    # Решение СЛУ
    x = A_inv @ b

    return x

A = np.array([[3, 2, 1],
[1, 6, 2],
[4, 1, 5]])
b = np.array([360, 300, 675])
x = minimal_residue_method(A,b)
print('The solution is:', x)

The solution is: [90. 15. 60.]


#### Метод сопряженных градиентов

Метод сопряженных градиентов - это итерационный метод решения задач оптимизации (главным образом, минимизации функций), который часто используется для решения задач квадратичной оптимизации и задач, близких к ним.

Основная идея метода заключается в использовании направления поиска на каждой итерации, которое является сопряженным к предыдущим направлениям относительно квадратичной формы, представленной в задаче. Это обеспечивает быструю сходимость метода и позволяет ему быть одним из самых эффективных методов решения задач квадратичной оптимизации.

Метод сопряженных градиентов особенно полезен в тех случаях, когда задача имеет большое количество переменных, так как он требует значительно меньше вычислений, чем другие методы, такие как метод Ньютона. Однако метод сопряженных градиентов может оказаться менее эффективным для задач с сильно неквадратичными функциями, поскольку в этих случаях метод может потребовать больше итераций для сходимости.

In [16]:
import numpy as np
def Print(A, B):
    for i in range(len(B)):
        for j in range(len(A[i])):
             print("\t{1:2.3f}{0}".format(" ", A[i][j]), end='')
        print("\t = \t{1:2.1f}".format(i + 1, B[i])) 
def Vektor_nevyzki(A,B,x):
    r=[0]*len(A)
    for i in range(len(A)):
        s=0
        for j in range(len(A)):
            s=s+A[i][j]*x[j]
        r[i]=B[i]-s
    return(r)    
def Ark(A,r):
    Ark=[0]*len(A)
    for i in range(len(A)):
        s=0
        for j in range(len(A)):
            s=s+A[i][j]*r[j]
        Ark[i]=s
    return(Ark) 


n=3
A = np.array([[3,2,1],[1,6,2],[4,1,5]])
B = np.array([360,300,675])

At=A.transpose()
A=np.dot(At,A)
B=np.dot(At,B)
itera=0
eps=0.001
X=[0]*len(A)
r=np.array(Vektor_nevyzki(A,B,X))
p=np.array([0]*len(A))
p=r
while abs(sum(r))>eps:
    alfa=np.dot(r, p)/np.dot(np.dot(A,p),p)
    X=X+np.dot(alfa,p)
    r=r-alfa*np.dot(A,p)
    b=-(np.dot(np.dot(A,p), r)/np.dot(np.dot(A,p),p))
    p=r+b*p
    itera=itera+1
print("результат метод сопряженных градиентов")
print(X)


результат метод сопряженных градиентов
[89.99903365 15.00154118 59.99928486]


7.Определить в каком из узлов АД температура в установившемся режиме
наибольшая, в каком — наименьшая. Расчет провести при номинальном входном
напряжении, в этом случае вектор потерь P для АД МТН111-6 имеет вид
P=(272 , 25 103 ,95 312 ,567 70 , 567 332, 75 127 , 05 0 0) . Для решения системы линейных уравнений использовать заработанные программы.


In [8]:
import numpy as np
def gaussian(A, b):
    '''
    Приминает на вход две переменные:
    A - матрица коэффициентов - квадратный двумерный numpy array типа float
    b - вектор свободных членов - numpy array типа float
    Вовращает вектор решений
    '''

    # составляем расширенную матрицу системы
    reshaped_b = b.reshape((len(b), 1))
    A = np.hstack((A, reshaped_b))
    
    # приводим матрицу к треугольному виду
    # i - опорная строка
    # j - текущая строка (всегда меньше i)
    for i in range(len(A)):
        for j in range(i + 1, len(A)):
            A[j] -= A[i] * A[j][i] / A[i][i]
            
    # обратный ход
    x = np.array([0] * len(b), dtype=float) # вектор решений

    i = len(A) - 1
    while i >= 0:
        x[i] = (A[i][-1] - sum(x * A[i][0:-1])) / A[i][i] 
        i -= 1
        
    return x

A_list =[[
    [34.27, 0, -22.22, 0, -12.05, 0, 0, 0],
    [0, 45.57, 0, -26.05, 0, -19.52, 0, 0],
    [-22.22, 0, 66.22, -4, 0, 0, 0, -40],
    [0, -26.05, -4, 39.49, 0, 0, -8.333, -1.103],
    [-12.05, 0, 0, 0, 22.05, 0, -10, 0],
    [0, -19.52, 0, 0, 0, 33.81, -14.29, 0],
    [0, 0, 0, -8.333, -10, -14.29, 39.29, -6.667],
    [0, 0, -40, -1.103, 0, 0, -6.667, 69.77]]]          
          
b_list = [ [272.25, 103.95, 312.567, 70.567, 332.75, 127.05, 0, 0] ]
for A in A_list:
    A = np.array(A, dtype=float)
    
for b in b_list:
    b = np.array(b, dtype=float)
for i in range(len(A_list)):
    ot=gaussian(A, b)
    print('Решение методом Гаусса:')
    print(ot, '\n')
    print('min:',np.min(ot), 'номер узла min',np.where(ot==np.min(ot))[0][0]+1)
    print('max:',np.max(ot), 'номер узла max',np.where(ot==np.max(ot))[0][0]+1)

Решение методом Гаусса:
[ 97.99137739 106.99708296  77.2647609  102.66911822 113.6175532
 107.44756869  99.17209504  55.39651471] 

min: 55.396514705778486 номер узла min 8
max: 113.61755319772048 номер узла max 5


In [9]:
import math
import copy
 
a = [[34.27, 0, -22.22, 0, -12.05, 0, 0, 0],
    [0, 45.57, 0, -26.05, 0, -19.52, 0, 0],
    [-22.22, 0, 66.22, -4, 0, 0, 0, -40],
    [0, -26.05, -4, 39.49, 0, 0, -8.333, -1.103],
    [-12.05, 0, 0, 0, 22.05, 0, -10, 0],
    [0, -19.52, 0, 0, 0, 33.81, -14.29, 0],
    [0, 0, 0, -8.333, -10, -14.29, 39.29, -6.667],
    [0, 0, -40, -1.103, 0, 0, -6.667, 69.77]]
     
b = [272.25, 103.95, 312.567, 70.567, 332.75, 127.05, 0, 0]
 
# Проверка матрицы коэффициентов на корректность
def isCorrectArray(a):
    for row in range(0, len(a)):
        if( len(a[row]) != len(b) ):
            print('Не соответствует размерность')
            return False
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True
 
# Условие завершения программы на основе вычисления
# расстояния между соответствующими элементами соседних
# итераций в методе решения 
def isNeedToComplete(x_old, x_new):
    eps = 0.0001
    sum_up = 0
    sum_low = 0
    for k in range(0, len(x_old)):
        sum_up += ( x_new[k] - x_old[k] ) ** 2
        sum_low += ( x_new[k] ) ** 2
        
    return math.sqrt( sum_up / sum_low ) < eps
 
# Процедура решения
def solution(a, b):
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
    else:
        count = len(b) # количество корней
        
        x = [1 for k in range(0, count) ] # начальное приближение корней
        
        nIter = 0  # подсчет количества итераций
        MAXI = 100    # максимально допустимое число итераций
        while( nIter < MAXI ):
 
            x_prev = copy.deepcopy(x)
            
            for k in range(0, count):
                S = 0
                for j in range(0, count):
                    if( j != k ): S = S + a[k][j] * x[j] 
                x[k] = b[k]/a[k][k] - S / a[k][k]
            
            if isNeedToComplete(x_prev, x) : # проверка на выход
                break
              
            nIter += 1
 
        print('Количество итераций на решение: ', nIter)
        
        return x    
                
    
# MAIN - блок программмы
ot= solution(a, b) 
print( 'Решение методом итерации: ', ot, '\n') # Вызываем процедуру решение
print('min:',np.min(ot), 'номер узла min',np.where(ot==np.min(ot))[0][0]+1)
print('max:',np.max(ot), 'номер узла max',np.where(ot==np.max(ot))[0][0]+1)

Количество итераций на решение:  87
Решение методом итерации:  [97.89279367008966, 106.85173778072908, 77.1842732312349, 102.53657050601056, 113.50624339150983, 107.31012736912201, 99.05477473344503, 55.337063919527075] 

min: 55.337063919527075 номер узла min 8
max: 113.50624339150983 номер узла max 5


In [36]:
import numpy as np

def seidel(m, b, eps):
    n = len(m)
    r = range(n)
    x = [0 for i in r]
    conv = False
    while not conv:
        p = x.copy()
        for i in r:
            var = sum(m[i][j] * x[j] for j in range(i))
            var += sum(m[i][j] * p[j] for j in range(i+1, n))
            x[i] = (b[i] - var) / m[i][i]
        
        conv = sum((x[i]-p[i])**2 for i in r) < eps
    return x
a = [[34.27, 0, -22.22, 0, -12.05, 0, 0, 0],
    [0, 45.57, 0, -26.05, 0, -19.52, 0, 0],
    [-22.22, 0, 66.22, -4, 0, 0, 0, -40],
    [0, -26.05, -4, 39.49, 0, 0, -8.333, -1.103],
    [-12.05, 0, 0, 0, 22.05, 0, -10, 0],
    [0, -19.52, 0, 0, 0, 33.81, -14.29, 0],
    [0, 0, 0, -8.333, -10, -14.29, 39.29, -6.667],
    [0, 0, -40, -1.103, 0, 0, -6.667, 69.77]]
     
b = [272.25, 103.95, 312.567, 70.567, 332.75, 127.05, 0, 0]

eps = 0.0001
ot=seidel(a, b, eps)
print('Решение методом Зейделя: ', ot, '\n')
print('min:',np.min(ot), 'номер узла min',np.where(ot==np.min(ot))[0][0]+1)
print('max:',np.max(ot), 'номер узла max',np.where(ot==np.max(ot))[0][0]+1)

Решение методом Зейделя:  [97.9544870384817, 106.94269431554793, 77.23464214885985, 102.61951842784667, 113.57590070174456, 107.39613769805669, 99.12819339749423, 55.37426802581916] 

min: 55.37426802581916  max: 113.57590070174456


In [39]:
import numpy as np
from numpy.linalg import norm
from math import *

def steepest_descent_manual(A, b, x_0, max_iters=1000, tol=1e-7, verbose=0):
    x_old = x_0.copy()
    it = 0
    while it < max_iters and norm(A@x_old - b) > tol:
        grad = np.transpose(A.dot(x_old)) - b
        alpha = np.inner(grad, grad) / (np.inner(np.transpose(np.vstack(A).dot(grad)), grad))
        x_new = x_old - alpha * grad
        it += 1
        if verbose >= 1:
            print("Iteration {}: x_old = {}, x_new = {}, error = {}".format(it, x_old, x_new, norm(Ax_old-b)))
        x_old = x_new
    return x_old
A = np.array([[34.27, 0, -22.22, 0, -12.05, 0, 0, 0],
    [0, 45.57, 0, -26.05, 0, -19.52, 0, 0],
    [-22.22, 0, 66.22, -4, 0, 0, 0, -40],
    [0, -26.05, -4, 39.49, 0, 0, -8.333, -1.103],
    [-12.05, 0, 0, 0, 22.05, 0, -10, 0],
    [0, -19.52, 0, 0, 0, 33.81, -14.29, 0],
    [0, 0, 0, -8.333, -10, -14.29, 39.29, -6.667],
    [0, 0, -40, -1.103, 0, 0, -6.667, 69.77]])
b = np.array([272.25, 103.95, 312.567, 70.567, 332.75, 127.05, 0, 0])
x_0 = np.ones(8)
ot=steepest_descent_manual(A,b,x_0)
print("Решение Метод наискорейшего спуска ",ot, '\n')
print('min:',np.min(ot), 'номер узла min',np.where(ot==np.min(ot))[0][0]+1)
print('max:',np.max(ot), 'номер узла max',np.where(ot==np.max(ot))[0][0]+1)

Решение Метод наискорейшего спуска  [ 97.99137738 106.99708294  77.26476089 102.66911819 113.61755318
 107.44756867  99.17209502  55.3965147 ] 

min: 55.39651469564261  max: 113.61755317751651


In [10]:
import numpy as np

def minimal_residue_method(A, b):
    # Проверка, является ли система уравнений квадратной и не единичной.
    if len(A) != len(b):
        raise ValueError('Input arrays must be square and non-singular.')

    # Вычисляем обратную матрицу A
    A_inv = np.linalg.inv(A)

    # Решение СЛУ
    x = A_inv @ b

    return x

A = np.array([[34.27, 0, -22.22, 0, -12.05, 0, 0, 0],
    [0, 45.57, 0, -26.05, 0, -19.52, 0, 0],
    [-22.22, 0, 66.22, -4, 0, 0, 0, -40],
    [0, -26.05, -4, 39.49, 0, 0, -8.333, -1.103],
    [-12.05, 0, 0, 0, 22.05, 0, -10, 0],
    [0, -19.52, 0, 0, 0, 33.81, -14.29, 0],
    [0, 0, 0, -8.333, -10, -14.29, 39.29, -6.667],
    [0, 0, -40, -1.103, 0, 0, -6.667, 69.77]])
b = np.array([272.25, 103.95, 312.567, 70.567, 332.75, 127.05, 0, 0])
ot = minimal_residue_method(A,b)
print('Решение Метод минимальных невязок:', ot, '\n')
print('min:',np.min(ot), 'номер узла min',np.where(ot==np.min(ot))[0][0]+1)
print('max:',np.max(ot), 'номер узла max',np.where(ot==np.max(ot))[0][0]+1)

Решение Метод минимальных невязок: [ 97.99137739 106.99708296  77.2647609  102.66911822 113.6175532
 107.44756869  99.17209504  55.39651471] 

min: 55.396514705778486 номер узла min 8
max: 113.6175531977205 номер узла max 5


In [9]:
import numpy as np
def Print(A, B):
    for i in range(len(B)):
        for j in range(len(A[i])):
             print("\t{1:2.3f}{0}".format(" ", A[i][j]), end='')
        print("\t = \t{1:2.1f}".format(i + 1, B[i])) 
def Vektor_nevyzki(A,B,x):
    r=[0]*len(A)
    for i in range(len(A)):
        s=0
        for j in range(len(A)):
            s=s+A[i][j]*x[j]
        r[i]=B[i]-s
    return(r)    
def Ark(A,r):
    Ark=[0]*len(A)
    for i in range(len(A)):
        s=0
        for j in range(len(A)):
            s=s+A[i][j]*r[j]
        Ark[i]=s
    return(Ark) 


A = np.array([[34.27, 0, -22.22, 0, -12.05, 0, 0, 0],
    [0, 45.57, 0, -26.05, 0, -19.52, 0, 0],
    [-22.22, 0, 66.22, -4, 0, 0, 0, -40],
    [0, -26.05, -4, 39.49, 0, 0, -8.333, -1.103],
    [-12.05, 0, 0, 0, 22.05, 0, -10, 0],
    [0, -19.52, 0, 0, 0, 33.81, -14.29, 0],
    [0, 0, 0, -8.333, -10, -14.29, 39.29, -6.667],
    [0, 0, -40, -1.103, 0, 0, -6.667, 69.77]])
B = np.array([272.25, 103.95, 312.567, 70.567, 332.75, 127.05, 0, 0])

At=A.transpose()
A=np.dot(At,A)
B=np.dot(At,B)
itera=0
eps=0.001
X=[0]*len(A)
r=np.array(Vektor_nevyzki(A,B,X))
p=np.array([0]*len(A))
p=r
while abs(sum(r))>eps:
    alfa=np.dot(r, p)/np.dot(np.dot(A,p),p)
    X=X+np.dot(alfa,p)
    r=r-alfa*np.dot(A,p)
    b=-(np.dot(np.dot(A,p), r)/np.dot(np.dot(A,p),p))
    p=r+b*p
    itera=itera+1
print("результат метод сопряженных градиентов")
ot=X
print('min:',np.min(ot), 'номер узла min',np.where(ot==np.min(ot))[0][0]+1)
print('max:',np.max(ot), 'номер узла max',np.where(ot==np.max(ot))[0][0]+1)

результат метод сопряженных градиентов
min: 55.39651490343804 номер узла min 8
max: 113.6175531918864 номер узла max 5


11.Решить тестовую СЛАУ Ax=b,где $$\begin{equation*}

\begin{equation*}
A_{i,j} = 
 \begin{cases}
   2n, &\text{i=j}\\
   1, &\text{i$\ne$ j}
 \end{cases} , b_i=\frac {n(n+1)}{2}+i(2n-1), i=1,...,n
\end{equation*}

In [32]:
def gaussian(A, b):
    '''
    Приминает на вход две переменные:
    A - матрица коэффициентов - квадратный двумерный numpy array типа float
    b - вектор свободных членов - numpy array типа float
    Вовращает вектор решений
    '''

    # составляем расширенную матрицу системы
    reshaped_b = b.reshape((len(b), 1))
    A = np.hstack((A, reshaped_b))
    
    # приводим матрицу к треугольному виду
    # i - опорная строка
    # j - текущая строка (всегда меньше i)
    for i in range(len(A)):
        for j in range(i + 1, len(A)):
            A[j] -= A[i] * A[j][i] / A[i][i]
            
    # обратный ход
    x = np.array([0] * len(b), dtype=float) # вектор решений

    i = len(A) - 1
    while i >= 0:
        x[i] = (A[i][-1] - sum(x * A[i][0:-1])) / A[i][i] 
        i -= 1
        
    return x    

n=6
m=6
A=[]
for i in range(n):
    A.append([0]*(m))
for i in range(n):
    for j in range(m):
        if i==j:
            A[i][j]=2*n
        else:
            A[i][j]=1
b=[]
for i in range(n):
    b.append(0)
for i in range(n):
    b[i]=((n*n+n)/2)+2*n*(i+1)-(i+1)

def Print(A, b):
    for i in range(len(b)):
        for j in range(len(A[i])):
             print("\t{1:2.3f}{0}".format(" ", A[i][j]), end='')
        print("\t = \t{1:2.1f}".format(i + 1, b[i]))
print("Введенная задача")
A=np.array(A)
b=np.array(b)
print('A',A)
print('b',b)
 
print('Решение методом Гаусса:')
print(gaussian(A, b), '\n')

Введенная задача
A [[12  1  1  1  1  1]
 [ 1 12  1  1  1  1]
 [ 1  1 12  1  1  1]
 [ 1  1  1 12  1  1]
 [ 1  1  1  1 12  1]
 [ 1  1  1  1  1 12]]
b [32. 43. 54. 65. 76. 87.]
Решение методом Гаусса:
[1. 2. 3. 4. 5. 6.] 



In [35]:
# Проверка матрицы коэффициентов на корректность
def isCorrectArray(a):
    for row in range(0, len(a)):
        if( len(a[row]) != len(b) ):
            print('Не соответствует размерность')
            return False
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True
 
# Условие завершения программы на основе вычисления
# расстояния между соответствующими элементами соседних
# итераций в методе решения 
def isNeedToComplete(x_old, x_new):
    eps = 0.0001
    sum_up = 0
    sum_low = 0
    for k in range(0, len(x_old)):
        sum_up += ( x_new[k] - x_old[k] ) ** 2
        sum_low += ( x_new[k] ) ** 2
        
    return math.sqrt( sum_up / sum_low ) < eps
 
# Процедура решения
def solution(a, b):
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
    else:
        count = len(b) # количество корней
        
        x = [1 for k in range(0, count) ] # начальное приближение корней
        
        nIter = 0  # подсчет количества итераций
        MAXI = 100    # максимально допустимое число итераций
        while( nIter < MAXI ):
 
            x_prev = copy.deepcopy(x)
            
            for k in range(0, count):
                S = 0
                for j in range(0, count):
                    if( j != k ): S = S + a[k][j] * x[j] 
                x[k] = b[k]/a[k][k] - S / a[k][k]
            
            if isNeedToComplete(x_prev, x) : # проверка на выход
                break
              
            nIter += 1
 
        print('Количество итераций на решение: ', nIter)
        
        return x    
                
    


n=5
m=5
A=[]
for i in range(n):
    A.append([0]*(m))
for i in range(n):
    for j in range(m):
        if i==j:
            A[i][j]=2*n
        else:
            A[i][j]=1
b=[]
for i in range(n):
    b.append(0)
for i in range(n):
    b[i]=((n*n+n)/2)+2*n*(i+1)-(i+1)

def Print(A, b):
    for i in range(len(b)):
        for j in range(len(A[i])):
             print("\t{1:2.3f}{0}".format(" ", A[i][j]), end='')
        print("\t = \t{1:2.1f}".format(i + 1, b[i]))
print("Введенная задача")
A=np.array(A)
b=np.array(b)
print('A',A)
print('b',b)

# MAIN - блок программмы
ot= solution(A, b) 
print( 'Решение методом итерации: ', ot, '\n') # Вызываем процедуру решение

Введенная задача
A [[10  1  1  1  1]
 [ 1 10  1  1  1]
 [ 1  1 10  1  1]
 [ 1  1  1 10  1]
 [ 1  1  1  1 10]]
b [24. 33. 42. 51. 60.]
Количество итераций на решение:  5
Решение методом итерации:  [0.9999951804068985, 2.000000555260176, 3.0000020769483666, 4.000001241574326, 5.000000094581023] 



In [2]:
import numpy as np
def seidel(m, b, eps):
    n = len(m)
    r = range(n)
    x = [0 for i in r]
    conv = False
    while not conv:
        p = x.copy()
        for i in r:
            var = sum(m[i][j] * x[j] for j in range(i))
            var += sum(m[i][j] * p[j] for j in range(i+1, n))
            x[i] = round((b[i] - var) / m[i][i])
        
        conv = sum((x[i]-p[i])**2 for i in r) < eps
    return x
n=5
m=5
A=[]
for i in range(n):
    A.append([0]*(m))
for i in range(n):
    for j in range(m):
        if i==j:
            A[i][j]=2*n
        else:
            A[i][j]=1
b=[]
for i in range(n):
    b.append(0)
for i in range(n):
    b[i]=((n*n+n)/2)+2*n*(i+1)-(i+1)

def Print(A, b):
    for i in range(len(b)):
        for j in range(len(A[i])):
             print("\t{1:2.3f}{0}".format(" ", A[i][j]), end='')
        print("\t = \t{1:2.1f}".format(i + 1, b[i]))
print("Введенная задача")
A=np.array(A)
b=np.array(b)
print('A',A)
print('b',b)


eps = 0.0001

print('Решение Зейделя',seidel(A, b, eps))

Введенная задача
A [[10  1  1  1  1]
 [ 1 10  1  1  1]
 [ 1  1 10  1  1]
 [ 1  1  1 10  1]
 [ 1  1  1  1 10]]
b [24. 33. 42. 51. 60.]
Решение Зейделя [1, 2, 3, 4, 5]


In [4]:
import numpy as np
from numpy.linalg import norm
from math import *

def steepest_descent_manual(A, b, x_0, max_iters=1000, tol=1e-7, verbose=0):
    x_old = x_0.copy()
    it = 0
    while it < max_iters and norm(A@x_old - b) > tol:
        grad = np.transpose(A.dot(x_old)) - b
        alpha = np.inner(grad, grad) / (np.inner(np.transpose(np.vstack(A).dot(grad)), grad))
        x_new = x_old - alpha * grad
        it += 1
        if verbose >= 1:
            print("Iteration {}: x_old = {}, x_new = {}, error = {}".format(it, x_old, x_new, norm(Ax_old-b)))
        x_old = x_new
    return x_old


n=5
m=5
A=[]
for i in range(n):
    A.append([0]*(m))
for i in range(n):
    for j in range(m):
        if i==j:
            A[i][j]=2*n
        else:
            A[i][j]=1
b=[]
for i in range(n):
    b.append(0)
for i in range(n):
    b[i]=((n*n+n)/2)+2*n*(i+1)-(i+1)

def Print(A, b):
    for i in range(len(b)):
        for j in range(len(A[i])):
             print("\t{1:2.3f}{0}".format(" ", A[i][j]), end='')
        print("\t = \t{1:2.1f}".format(i + 1, b[i]))
print("Введенная задача")
A=np.array(A)
b=np.array(b)
print('A',A)
print('b',b)
x_0 = np.ones(n)
ot=steepest_descent_manual(A,b,x_0)
print("Решение Метод наискорейшего спуска ",ot, '\n')

Введенная задача
A [[10  1  1  1  1]
 [ 1 10  1  1  1]
 [ 1  1 10  1  1]
 [ 1  1  1 10  1]
 [ 1  1  1  1 10]]
b [24. 33. 42. 51. 60.]
Решение Метод наискорейшего спуска  [1. 2. 3. 4. 5.] 



In [5]:
import numpy as np

def minimal_residue_method(A, b):
    # Проверка, является ли система уравнений квадратной и не единичной.
    if len(A) != len(b):
        raise ValueError('Input arrays must be square and non-singular.')

    # Вычисляем обратную матрицу A
    A_inv = np.linalg.inv(A)

    # Решение СЛУ
    x = A_inv @ b

    return x

n=5
m=5
A=[]
for i in range(n):
    A.append([0]*(m))
for i in range(n):
    for j in range(m):
        if i==j:
            A[i][j]=2*n
        else:
            A[i][j]=1
b=[]
for i in range(n):
    b.append(0)
for i in range(n):
    b[i]=((n*n+n)/2)+2*n*(i+1)-(i+1)

def Print(A, b):
    for i in range(len(b)):
        for j in range(len(A[i])):
             print("\t{1:2.3f}{0}".format(" ", A[i][j]), end='')
        print("\t = \t{1:2.1f}".format(i + 1, b[i]))
print("Введенная задача")
A=np.array(A)
b=np.array(b)
print('A',A)
print('b',b)
ot = minimal_residue_method(A,b)
print('Решение Метод минимальных невязок:', ot, '\n')

Введенная задача
A [[10  1  1  1  1]
 [ 1 10  1  1  1]
 [ 1  1 10  1  1]
 [ 1  1  1 10  1]
 [ 1  1  1  1 10]]
b [24. 33. 42. 51. 60.]
Решение Метод минимальных невязок: [1. 2. 3. 4. 5.] 



In [17]:
import numpy as np
def Print(A, B):
    for i in range(len(B)):
        for j in range(len(A[i])):
             print("\t{1:2.3f}{0}".format(" ", A[i][j]), end='')
        print("\t = \t{1:2.1f}".format(i + 1, B[i])) 
def Vektor_nevyzki(A,B,x):
    r=[0]*len(A)
    for i in range(len(A)):
        s=0
        for j in range(len(A)):
            s=s+A[i][j]*x[j]
        r[i]=B[i]-s
    return(r)    
def Ark(A,r):
    Ark=[0]*len(A)
    for i in range(len(A)):
        s=0
        for j in range(len(A)):
            s=s+A[i][j]*r[j]
        Ark[i]=s
    return(Ark) 
n=5
m=5
A=np.array([[0 for j in range(n)] for i in range(m)])
for i in range(n):
    for j in range(m):
        if i==j:
            A[i][j]=2*n
        else:
            A[i][j]=1
B=[0]*n
for i in range(n):
    B[i]=((n*n+n)/2)+2*n*(i+1)-(i+1)
print("Введенная задача")
print(A)
print(B)
itera=0
#At=A.transpose()
#A=np.dot(A,At)
#B=np.dot(At,B)
itera=0
eps=0.001
X=[0]*len(A)
r=np.array(Vektor_nevyzki(A,B,X))
p=np.array([0]*len(A))
p=r
while abs(sum(r))>eps:
    alfa=np.dot(r, p)/np.dot(np.dot(A,p),p)
    X=X+np.dot(alfa,p)
    r=r-alfa*np.dot(A,p)
    b=-(np.dot(np.dot(A,p), r)/np.dot(np.dot(A,p),p))
    p=r+b*p
    itera=itera+1
print("Итоговый результат", X)

Введенная задача
[[10  1  1  1  1]
 [ 1 10  1  1  1]
 [ 1  1 10  1  1]
 [ 1  1  1 10  1]
 [ 1  1  1  1 10]]
[24.0, 33.0, 42.0, 51.0, 60.0]
Итоговый результат [1. 2. 3. 4. 5.]
